<a href="https://colab.research.google.com/github/arunillo/5lab-python/blob/main/5python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Установка
!pip install aiohttp nest-asyncio -q
import nest_asyncio
nest_asyncio.apply()

import time
import threading
import multiprocessing
import asyncio
import aiohttp
import math
import random
from datetime import datetime
import concurrent.futures

#ЗАДАЧА 1: Синхронный калькулятор
def sync_calculate(operation, a, b, delay):
    time.sleep(delay)  # Имитация вычислений

    if operation == '+':
        result = a + b
    elif operation == '-':
        result = a - b
    elif operation == '*':
        result = a * b
    elif operation == '/':
        result = a / b if b != 0 else 'Ошибка: деление на ноль'
    else:
        result = 'Неизвестная операция'

    return result

def task1_sync_calculations():
    start_time = time.time()

    # Последовательное выполнение 4 операций
    results = []
    results.append(sync_calculate('+', 15, 25, 2))
    results.append(sync_calculate('-', 40, 18, 1))
    results.append(sync_calculate('*', 12, 8, 3))
    results.append(sync_calculate('/', 100, 5, 1))

    end_time = time.time()
    print(f"Задача 1: {end_time - start_time:.2f} сек")
    return results

 # ЗАДАЧА 2: Многопоточный загрузчик
def download_file(filename, size):
    download_time = size * 0.1  # 0.1 сек на МБ
    time.sleep(download_time)
    return filename

def task2_threaded_downloader():
    files = [("doc.pdf", 10), ("image.jpg", 5),
             ("video.mp4", 20), ("archive.zip", 15)]

    start_time = time.time()
    threads = []
    results = []

    # Создание и запуск потоков
    for filename, size in files:
        thread = threading.Thread(
            target=lambda f=filename, s=size: results.append(download_file(f, s))
        )
        threads.append(thread)
        thread.start()

    # Ожидание завершения
    for thread in threads:
        thread.join()

    end_time = time.time()
    print(f"Задача 2: {end_time - start_time:.2f} сек")
    return results

#  ЗАДАЧА 3: Многопроцессные вычисления
def calculate_factorial(n):
    return math.factorial(n)

def calculate_prime(n):
    if n < 2:
        return False
    return all(n % i != 0 for i in range(2, int(math.sqrt(n)) + 1))

def task3_multiprocess_calculations():
    calculations = [
        (calculate_factorial, 10000),
        (calculate_factorial, 8000),
        (calculate_prime, 10000019),
        (calculate_prime, 10000033)
    ]

    # Многопроцессное выполнение
    start_time = time.time()
    processes = []
    manager = multiprocessing.Manager()
    results = manager.list()

    for func, arg in calculations:
        p = multiprocessing.Process(
            target=lambda f=func, a=arg, r=results: r.append(f(a))
        )
        processes.append(p)
        p.start()

    for p in processes:
        p.join()

    multiprocess_time = time.time() - start_time

    # Синхронное выполнение для сравнения
    start_time = time.time()
    sync_results = []
    for func, arg in calculations:
        sync_results.append(func(arg))
    sync_time = time.time() - start_time

    print(f"Задача 3: Многопроцессное - {multiprocess_time:.2f} сек, "
          f"Синхронное - {sync_time:.2f} сек")
    return multiprocess_time, sync_time

#  ЗАДАЧА 4: Асинхронный веб-скрапер
async def fetch_url(session, url, name):
    try:
        async with session.get(url, timeout=aiohttp.ClientTimeout(total=10)) as response:
            content = await response.text()
            return (name, len(content), response.status)
    except Exception:
        return (name, 0, 0)

async def task4_async_scraper():
    urls = [
        ("https://httpbin.org/delay/1", "Сайт 1"),
        ("https://httpbin.org/delay/2", "Сайт 2"),
        ("https://httpbin.org/delay/1", "Сайт 3"),
        ("https://httpbin.org/delay/3", "Сайт 4")
    ]

    start_time = time.time()

    async with aiohttp.ClientSession() as session:
        tasks = [fetch_url(session, url, name) for url, name in urls]
        results = await asyncio.gather(*tasks)

    async_time = time.time() - start_time
    print(f"Задача 4: {async_time:.2f} сек")
    return async_time, results

#  ЗАДАЧА 5: Сравнение производительности
def io_task(name, duration):
    time.sleep(duration)
    return f"{name} completed"

async def async_io_task(name, duration):
    await asyncio.sleep(duration)
    return f"{name} completed"

def task5_performance_comparison():
    tasks = [("Task1", 2), ("Task2", 3), ("Task3", 1),
             ("Task4", 2), ("Task5", 1)]

    # 1. Синхронное выполнение
    start_time = time.time()
    sync_results = [io_task(name, duration) for name, duration in tasks]
    sync_time = time.time() - start_time

    # 2. Многопоточное выполнение
    start_time = time.time()
    threads = []
    thread_results = []
    for name, duration in tasks:
        thread = threading.Thread(
            target=lambda n=name, d=duration: thread_results.append(io_task(n, d))
        )
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()
    thread_time = time.time() - start_time

    # 3. Многопроцессное выполнение
    start_time = time.time()
    processes = []
    manager = multiprocessing.Manager()
    process_results = manager.list()
    for name, duration in tasks:
        p = multiprocessing.Process(
            target=lambda n=name, d=duration, r=process_results: r.append(io_task(n, d))
        )
        processes.append(p)
        p.start()
    for p in processes:
        p.join()
    process_time = time.time() - start_time

    # 4. Асинхронное выполнение
    async def run_async():
        async_tasks = [async_io_task(name, duration) for name, duration in tasks]
        return await asyncio.gather(*async_tasks)

    start_time = time.time()
    async_results = asyncio.run(run_async())
    async_time = time.time() - start_time

    print(f"Задача 5: Синхронное={sync_time:.2f}с, "
          f"Многопоточное={thread_time:.2f}с, "
          f"Многопроцессное={process_time:.2f}с, "
          f"Асинхронное={async_time:.2f}с")

    return sync_time, thread_time, process_time, async_time

#  ЗАДАЧА 6: Асинхронный планировщик
async def scheduled_task(name, priority, duration):
    await asyncio.sleep(duration)
    return f"Результат {name}"

async def task6_async_scheduler():
    tasks_with_priority = [
        ("Экстренная", 1, 1),
        ("Важная", 2, 2),
        ("Обычная A", 3, 3),
        ("Обычная B", 3, 2),
        ("Фоновая", 4, 5)
    ]

    # Сортировка по приоритету
    tasks_with_priority.sort(key=lambda x: x[1])

    # Семафор для ограничения одновременного выполнения (max 2 задачи)
    semaphore = asyncio.Semaphore(2)

    async def run_task_with_semaphore(name, priority, duration):
        async with semaphore:
            return await scheduled_task(name, priority, duration)

    start_time = time.time()

    tasks = [
        run_task_with_semaphore(name, priority, duration)
        for name, priority, duration in tasks_with_priority
    ]

    results = await asyncio.gather(*tasks)

    async_time = time.time() - start_time
    print(f"Задача 6: {async_time:.2f} сек")
    return results

#  ЗАДАЧА 7: Пул потоков
def process_data(item):
    process_time = random.uniform(0.5, 2.0)
    time.sleep(process_time)
    return item * 2

def task7_thread_pool():
    data = list(range(1, 11))
    results = {}

    for pool_size in [2, 4, 8]:
        start_time = time.time()

        with concurrent.futures.ThreadPoolExecutor(max_workers=pool_size) as executor:
            # Отправка задач в пул
            future_to_item = {executor.submit(process_data, item): item
                            for item in data}

            pool_results = []
            for future in concurrent.futures.as_completed(future_to_item):
                pool_results.append(future.result())

        execution_time = time.time() - start_time
        results[pool_size] = execution_time

        print(f"  Пул {pool_size} потоков: {execution_time:.2f} сек")

    return results

# ========== ОСНОВНОЙ ЗАПУСК ==========
def main():
    print("Запуск лабораторной работы:")
    print("-" * 30)

    # Задача 1
    results1 = task1_sync_calculations()

    # Задача 2
    results2 = task2_threaded_downloader()

    # Задача 3
    time3_mp, time3_sync = task3_multiprocess_calculations()

    # Задача 4
    time4, results4 = asyncio.run(task4_async_scraper())

    # Задача 5
    sync_t, thread_t, process_t, async_t = task5_performance_comparison()

    # Задача 6
    results6 = asyncio.run(task6_async_scheduler())

    # Задача 7
    print("\nЗадача 7: Пул потоков")
    results7 = task7_thread_pool()

    print("\n" + "=" * 40)
    print("Итоги:")
    print(f"1. Синхронный: {results1}")
    print(f"2. Многопоточный: {results2}")
    print(f"3. Многопроцессный: ускорение в {time3_sync/time3_mp:.1f} раз")
    print(f"4. Асинхронный веб: {time4:.1f} сек")
    print(f"5. Сравнение: async быстрее sync в {sync_t/async_t:.1f} раз")
    print(f"6. Планировщик: {len(results6)} задач")
    print(f"7. Пул потоков: {results7}")

if __name__ == "__main__":
    main()

Запуск лабораторной работы:
------------------------------
Задача 1: 7.00 сек
Задача 2: 2.00 сек
Задача 3: Многопроцессное - 0.06 сек, Синхронное - 0.01 сек
Задача 4: 0.22 сек
Задача 5: Синхронное=9.00с, Многопоточное=3.00с, Многопроцессное=3.04с, Асинхронное=3.00с
Задача 6: 9.01 сек

Задача 7: Пул потоков
  Пул 2 потоков: 8.80 сек
  Пул 4 потоков: 3.79 сек
  Пул 8 потоков: 1.97 сек

Итоги:
1. Синхронный: [40, 22, 96, 20.0]
2. Многопоточный: ['image.jpg', 'doc.pdf', 'archive.zip', 'video.mp4']
3. Многопроцессный: ускорение в 0.1 раз
4. Асинхронный веб: 0.2 сек
5. Сравнение: async быстрее sync в 3.0 раз
6. Планировщик: 5 задач
7. Пул потоков: {2: 8.80055046081543, 4: 3.786632537841797, 8: 1.9706506729125977}
